In [1]:
from tools import HopfieldNetwork
import numpy as np

In [2]:
for i in range(1, 10): 
    hn = HopfieldNetwork(1024)
    pattern_indices = [x for x in range(1, i+1)]
    hn.fit(pattern_indices=pattern_indices)

    distorted_memories = hn.distort_patterns(
        num_units=32,
        pattern_indices=pattern_indices
    )

    patterns_pred = hn.recall(inputs=distorted_memories)
    results = hn.evaluate(
        patterns_pred=patterns_pred,
        pattern_indices=pattern_indices
    )
    print(results)
